In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from src.extra.model import ModelDistribution
from src.extra.pprint import print_model
from src.core.core import reset_model

from src.greedy.greedy import greedy_algorithm
from src.greedy.resource_allocation_policy import SumPercentage
from src.greedy.server_selection_policy import SumResources
from src.greedy.task_prioritisation import UtilityDeadlinePerResource

from src.optimal.fixed_optimal import fixed_optimal
from src.core.fixed_task import FixedTask, SumSpeedPowFixedAllocationPriority

In [3]:
def print_results(result, tasks, servers):
    print(f'Social welfare: {result.social_welfare:.3f}, Percentage social welfare: {result.percentage_social_welfare:.3f}, Percentage tasks: {result.percentage_tasks_allocated:.3f}\n')
    # for server in servers:
    #    storage_usage = sum(task.required_storage for task in server.allocated_tasks) / server.storage_capacity
    #    computational_usage = sum(task.compute_speed for task in server.allocated_tasks) / server.computation_capacity
    #    bandwidth_usage = sum(task.loading_speed + task.sending_speed for task in server.allocated_tasks) / server.bandwidth_capacity
    #    print(f'{server.name} resouce usage - Stor: {storage_usage:.3f}, Comp: {computational_usage:.3f}, Band: {bandwidth_usage:.3f}')
    print(f'Server avg storage: {np.mean([sum(task.required_storage for task in server.allocated_tasks) / server.storage_capacity for server in servers]):3.2f}, '
          f'computation: {np.mean([sum(task.compute_speed for task in server.allocated_tasks) / server.computation_capacity for server in servers]):3.2f}, '
          f'bandwidth: {np.mean([sum(task.loading_speed + task.sending_speed for task in server.allocated_tasks) / server.bandwidth_capacity for server in servers]):3.2f}')
    
    avg_loading_speed = np.mean([task.loading_speed for task in tasks if task.running_server])
    avg_compute_speed = np.mean([task.compute_speed for task in tasks if task.running_server])
    avg_sending_speed = np.mean([task.sending_speed for task in tasks if task.running_server])
    print(f'\nTask avg loading speed: {avg_loading_speed:3.2f}, compute speed: {avg_compute_speed:3.2f}, sending speed: {avg_sending_speed:3.2f}')
    avg_loading_time = np.mean([task.required_storage / task.loading_speed for task in tasks if task.running_server])
    avg_compute_time = np.mean([task.required_computation / task.compute_speed for task in tasks if task.running_server])
    avg_sending_time = np.mean([task.required_results_data / task.sending_speed for task in tasks if task.running_server])
    print(f'Task avg loading time: {avg_loading_time:3.2f}, compute time: {avg_compute_time:3.2f}, sending time: {avg_sending_time:3.2f}')
    

def avg_resource(tasks, attribute):
    return np.mean([getattr(task, attribute) for task in tasks])

In [4]:
alibaba_model = ModelDistribution('../realistic.mdl', num_tasks=100, num_servers=10)

In [6]:
tasks, servers = alibaba_model.generate()
print(f'Task avg storage: {avg_resource(tasks, "required_storage"):3.2f}, '
      f'computation: {avg_resource(tasks, "required_computation"):3.2f}, '
      f'results data: {avg_resource(tasks, "required_results_data"):3.2f}')

fixed_tasks = [FixedTask(task, SumSpeedPowFixedAllocationPriority()) for task in tasks]
print(f'\nFixed task avg storage: {avg_resource(fixed_tasks, "required_storage"):3.2f}, '
      f'computation: {avg_resource(fixed_tasks, "required_computation"):3.2f}, '
      f'results data: {avg_resource(fixed_tasks, "required_results_data"):3.2f}')
print(f'Fixed task avg loading speed: {avg_resource(fixed_tasks, "loading_speed"):3.2f}, '
      f'compute speed: {avg_resource(fixed_tasks, "compute_speed"):3.2f}, '
      f'sending speed: {avg_resource(fixed_tasks, "sending_speed"):3.2f}')

foreknowledge_fixed_tasks = [FixedTask(task, SumSpeedPowFixedAllocationPriority(), resource_foreknowledge=True) 
                             for task in tasks]
print(f'\nForeknowledge fixed task avg storage: {avg_resource(foreknowledge_fixed_tasks, "required_storage"):3.2f}, '
      f'computation: {avg_resource(foreknowledge_fixed_tasks, "required_computation"):3.2f}, '
      f'results data: {avg_resource(foreknowledge_fixed_tasks, "required_results_data"):3.2f}')
print(f'Foreknowledge fixed task avg loading speed: {avg_resource(foreknowledge_fixed_tasks, "loading_speed"):3.2f}, '
      f'compute speed: {avg_resource(foreknowledge_fixed_tasks, "compute_speed"):3.2f}, '
      f'sending speed: {avg_resource(foreknowledge_fixed_tasks, "sending_speed"):3.2f}')

Task avg storage: 54.69, computation: 620.49, results data: 20.20

Fixed task avg storage: 146.10, computation: 2486.50, results data: 20.20
Fixed task avg loading speed: 41.21, compute speed: 74.50, sending speed: 9.22

Foreknowledge fixed task avg storage: 54.69, computation: 620.49, results data: 20.20
Foreknowledge fixed task avg loading speed: 11.79, compute speed: 21.26, sending speed: 9.22


In [7]:
result = greedy_algorithm(tasks, servers, UtilityDeadlinePerResource(), SumResources(), SumPercentage())
print_results(result, tasks, servers)

Social welfare: 820.040, Percentage social welfare: 0.764, Percentage tasks: 0.620

Server avg storage: 0.88, computation: 0.65, bandwidth: 0.86

Task avg loading speed: 4.32, compute speed: 17.85, sending speed: 2.61
Task avg loading time: 16.84, compute time: 29.13, sending time: 8.76


In [9]:
reset_model(tasks, servers)
result = fixed_optimal(fixed_tasks, servers, time_limit=4)
print_results(result, fixed_tasks, servers)

Social welfare: 489.340, Percentage social welfare: 0.456, Percentage tasks: 0.230

Server avg storage: 0.76, computation: 0.91, bandwidth: 0.93

Task avg loading speed: 15.43, compute speed: 63.04, sending speed: 4.87
Task avg loading time: 10.88, compute time: 47.09, sending time: 4.62


In [10]:
reset_model(tasks, servers)
result = fixed_optimal(foreknowledge_fixed_tasks, servers, time_limit=4)
print_results(result, foreknowledge_fixed_tasks, servers)

Social welfare: 697.250, Percentage social welfare: 0.649, Percentage tasks: 0.470

Server avg storage: 0.66, computation: 0.28, bandwidth: 0.99

Task avg loading speed: 5.89, compute speed: 8.89, sending speed: 4.64
Task avg loading time: 11.44, compute time: 45.35, sending time: 4.84
